# Analyse des vergangenen Sommers

To Do:
- Binder / Colab Links anpassen
_________

Dieses Notebook kann lokal oder **direkt im Browser** auf [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/meteotest/urban-heat-API-docs/add-heatdays-and-tropicalnights-analysis?labpath=python_data_analysis.ipynb) oder [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meteotest/urban-heat-API-docs/blob/add-heatdays-and-tropicalnights-analysis/python_data_analysis.ipynb) ausgeführt werden.

In [2]:
import requests 
import pandas as pd
import geopandas as gpd 
import matplotlib # for plotting
import folium
import branca
import math

In [3]:
# Auswahl an Zeitraum
# get data of specific period (i.e. a heatwave)
time_from = "2024-06-01T22:00:00Z" # startzeit
time_to = "2024-06-30T23:50:00Z" # endzeit

# Auswahl an Stationen
# Wenn alle Stationen berücksichtigt werden sollen, dann einfach die Liste leer lassen
station_name_subset = [
                        'Gurten Kulm'
                    , 'Breitenrain, Waffenweg'
                    , 'Zollikofen 3m'
                    ]

# hier ist eine Liste alle Stationen mit Name und ID:
# https://smart-urban-heat-map.ch/api/v2/stations

In [4]:
def get_stations() -> gpd.GeoDataFrame:
    response = requests.get(
            url=f"https://smart-urban-heat-map.ch/api/v2/stations"
    )
    stations = gpd.read_file(response.text)

    # calculate only at subset of stations 
    # (if a subset was defined)
    if station_name_subset != []:
        stations = stations.loc[stations['name'].isin(station_name_subset)] 
    
    stations.drop(["latestMeasurementDate"], axis=1, inplace=True)

    return stations

def get_station_analysis(time_from: str, time_to: str) -> pd.DataFrame:
    stations = get_stations()

    # check that time series are at least 83 % complete
    time_from_dt = pd.to_datetime(time_from)
    time_to_dt = pd.to_datetime(time_to)
    time_difference = time_to_dt - time_from_dt
    expected_values = round(time_difference.days * 24 * 6 * 0.83)
    
    stations['einschlaftemperatur'] = None
    
    for idx, station in stations.iterrows():
        station_id = station.stationId
        response = requests.get(url=f"https://smart-urban-heat-map.ch/api/v2/timeseries?stationId={station_id}&timeFrom={time_from}&timeTo={time_to}")
        
        payload = response.json()

        if payload["values"] is None or not len(payload["values"]): 
            stations = stations.drop(idx)
            continue
            
        if len(payload["values"]) < expected_values:
            stations = stations.drop(idx)
            continue
            
        df = pd.DataFrame(payload["values"])

        df["dateObserved"] = pd.to_datetime(df["dateObserved"])
        df["dateObserved"] = df["dateObserved"].dt.tz_convert("Europe/Zurich")
        
        # hier werden die Hitzetage und Tropennächte berechnet
        einschlaftemperatur = calc_einschlaftemperatur(df)
        
        stations.loc[idx, 'einschlaftemperatur'] = einschlaftemperatur

    return stations

def calc_einschlaftemperatur(df: pd.DataFrame) -> float:
    sleep_time_mask = (df['dateObserved'].dt.hour >= 22) & (df['dateObserved'].dt.hour < 23)
    sleep_time = df.loc[sleep_time_mask]

    nightly_sleep_temperatures = (
        sleep_time.groupby(sleep_time['dateObserved'].dt.date)['temperature'].mean()
    )

    return nightly_sleep_temperatures.mean()


# TO DO: Calculate UHI for selected stations (make the stations definable by user)
# compared to 'Zollikofen 3m' as the rural reference station

# TO DO: UHI function
# def calc_uhi(df: pd.DataFrame) -> float:
#     # dummy definition of UHI
#      uhi = float(2.0)
#      return uhi

In [5]:
stations = get_stations()
stations

,name,stationId,geometry
2,"Breitenrain, Waffenweg",11003,POINT (7.45192 46.96173)
72,Gurten Kulm,11075,POINT (7.43971 46.91833)
116,Zollikofen 3m,11119,POINT (7.46407 46.99079)


In [6]:
# run analysis
station_analysis = get_station_analysis(time_from, time_to)
station_analysis

,name,stationId,geometry,einschlaftemperatur
2,"Breitenrain, Waffenweg",11003,POINT (7.45192 46.96173),18.458032
72,Gurten Kulm,11075,POINT (7.43971 46.91833),15.826256
116,Zollikofen 3m,11119,POINT (7.46407 46.99079),16.913167


In [7]:
# Einschlaftemperatur Map
m = folium.Map(location=[station_analysis.geometry.y.mean(), station_analysis.geometry.x.mean()], zoom_start=13, tiles="CartoDB positron")

# Add a fixed title to the map
title_html = f'''
     <div style="position: fixed; 
     top: 20px; left: 100px; width: 25%; height: 45px; 
     background-color: #F0F0F0; border: 1px solid black; z-index: 9999; font-size: 14px; font-weight: bold;">
     Einschlaftemperatur zwischen 22:00 und 23:00<br> ({pd.to_datetime(time_from).strftime('%d.%m.%Y')} bis {pd.to_datetime(time_to).strftime('%d.%m.%Y')})
     </div>
     '''
m.get_root().html.add_child(folium.Element(title_html))

colormap = branca.colormap.linear.YlOrRd_09
einschlaftemperatur = station_analysis.einschlaftemperatur.values

# Define colourmap range depending on values of 'einschlaftemperatur
vmin = math.floor(einschlaftemperatur.min())
vmax = math.ceil(einschlaftemperatur.max())

# Define the colormap with the specified range
colormap = branca.colormap.linear.YlOrRd_09.scale(vmin, vmax)

# Convert to step colormap with a specified number of steps
n_steps = int((vmax - vmin) / 0.5)  # Define the number of steps
colormap = colormap.to_step(n_steps)

# colormap = colormap.scale(0, einschlaftemperatur).to_step(einschlaftemperatur) 
colormap.caption = "Mittlere Einschlaftemperatur"
colormap.add_to(m)

# plot each station temperature
for idx, station in station_analysis.iterrows():
    color = colormap(station.einschlaftemperatur)
    # text with temperature value
    folium.Marker(
        location=(station.geometry.y, station.geometry.x),
        icon=folium.DivIcon(
            html=f'<div style="font-size: 10pt; color: {color}; text-shadow: -1px -1px 0 #D3D3D3, 1px -1px 0 #D3D3D3, -1px 1px 0 #D3D3D3, 1px 1px 0 #D3D3D3;">{station.einschlaftemperatur:.1f}°C</div>'
            ),
        tooltip=f"{station['name']}: Einschlaftemperatur {station.einschlaftemperatur:.1f} °C",
    ).add_to(m)

# show map
m

In [8]:
# "Urban Heat Island" Effekt Grafik
